# Stream Analysis

## Trending Topics

Assuming you receive Reddit comments as a live stream, build a Spark streaming application that will consume the stream and determine what topics are trending over a particular window of time.

Data was streamed using the streamer.py script which can be found in the scripts directory of this repo.

Note: The code in this notebook borrows heavily from the from the spark-examples repo in the USF-BigData GitHub https://github.com/USF-BigData/spark-examples

## Approach 1: Frequent words

In [51]:
from pyspark.streaming import StreamingContext

# The second parameter is the number of seconds between microbatches
ssc = StreamingContext(sc, 1)

# Required to be able to do state updates
ssc.checkpoint("/bigdata/amkaddoura/checkpoint")

In [52]:
# Stream comments from orion05
sock = ssc.socketTextStream("orion05", 13999)

In [53]:
import json

def get_text(comment_json):
    return comment_json["body"]

In [54]:
from nltk.corpus import stopwords

# Get a set of common "stop words" which are common English words that are not trending topics
stop_words = set(stopwords.words("english"))

In [55]:
import string

def split_comment(comment):
    words = comment.split()
    
    # Source for removing punctuation: https://stackoverflow.com/a/266162
    translator = str.maketrans('', '', string.punctuation)
    words = [word.translate(translator) for word in words]
    
    # Convert all letters to lowercase
    words = [word.lower() for word in words]
    
    # Source for removing stop words: https://stackoverflow.com/a/5486535
    words = [word for word in words if word not in stop_words]
    
    return words

In [56]:
json_objects = sock.map(lambda line: json.loads(line))
text_data = json_objects.flatMap(lambda x: [x.get("body")])
filtered_comments = text_data.filter(lambda comment: comment != "[deleted]")
filtered_comments_words = filtered_comments.flatMap(split_comment)

In [57]:
from operator import itemgetter

def print_frequent_words(rdd):
    top_words = sorted(rdd.collect(), key=itemgetter(1), reverse=True)[:10]
    print(top_words)

In [58]:
word_counts = filtered_comments_words.map(lambda x: (x, 1))
total_counts = word_counts.reduceByKey(lambda x, y: x + y)

total_counts.foreachRDD(print_frequent_words)

In [59]:
# Running this will start listening
ssc.start()

[('', 33), ('like', 28), ('im', 24), ('get', 18), ('dont', 17), ('one', 17), ('think', 14), ('good', 14), ('way', 13), ('first', 12)]
[('like', 17), ('good', 12), ('one', 12), ('would', 11), ('people', 11), ('well', 9), ('get', 8), ('im', 8), ('dont', 8), ('really', 8)]
[('', 42), ('like', 32), ('would', 29), ('get', 28), ('one', 25), ('dont', 19), ('really', 18), ('people', 15), ('thats', 14), ('two', 13)]
[('like', 15), ('really', 11), ('', 10), ('dont', 10), ('would', 8), ('people', 8), ('music', 7), ('im', 7), ('someone', 7), ('one', 7)]
[('like', 23), ('', 16), ('would', 13), ('im', 13), ('dont', 13), ('think', 12), ('really', 10), ('get', 9), ('even', 9), ('people', 9)]
[('like', 18), ('get', 12), ('one', 12), ('dont', 9), ('would', 8), ('see', 8), ('say', 7), ('know', 7), ('better', 7), ('', 6)]
[('like', 24), ('', 16), ('would', 15), ('one', 15), ('dont', 12), ('get', 10), ('im', 10), ('2', 9), ('much', 9), ('want', 9)]
[('like', 28), ('im', 23), ('dont', 23), ('one', 22), ('wo

In [60]:
# Note: you need the stopSparkContext=False, otherwise your driver will die and you'll have to restart Jupyter
ssc.stop(stopSparkContext=False)

[('', 23), ('would', 17), ('ukip', 15), ('even', 14), ('like', 13), ('much', 11), ('see', 11), ('think', 10), ('one', 10), ('dont', 10)]
[]


### Analysis

The approach of finding the most frequent words in the comments has potential, but there is too much noise in the comment text data to find the signal of the trending topics. The noise is common words which aren't stop words but aren't topics of discussion. The other problem is that a trending topic is not necessarily a single word. This approach could be improved upon by adding stemming of words, but even then there are too many frequent words which are not trending topics. My next approach will be to look for named entities which are more likely to be trending topics.

## Approach 2: Named entities

In [61]:
import nltk

nltk.download('words')
nltk.download('maxent_ne_chunker')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package words to /home/amkaddoura/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /home/amkaddoura/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/amkaddoura/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [66]:
from nltk import ne_chunk, pos_tag, word_tokenize
from nltk.tree import Tree

def find_entities(comments):
    entities = []
    chunked_comment = ne_chunk(pos_tag(word_tokenize(comments)))
    
    for subtree in chunked_comment:
        if type(subtree) == Tree:
            entity_name = " ".join([word for word, tag in subtree.leaves()])
            entities.append(entity_name)
    
    return entities

def find_trending_topics(comments):
    entities = comments.flatMap(find_entities)
    entity_counts = entities.map(lambda entity: (entity, 1)).reduceByKey(lambda x, y: x + y)
    trending_topics = entity_counts.sortBy(lambda x: x[1], ascending=False).take(10)
    
    print(trending_topics)
        
    return trending_topics

In [67]:
from pyspark.streaming import StreamingContext

ssc = StreamingContext(sc, 60)
ssc.checkpoint("/bigdata/amkaddoura/checkpoint")
sock = ssc.socketTextStream("orion05", 13999)

import json

def get_text(comment_json):
    return comment_json["body"]

json_objects = sock.map(lambda line: json.loads(line))
text_data = json_objects.flatMap(lambda row: [row.get("body")])
filtered_comments = text_data.filter(lambda comment: comment != "deleted")

filtered_comments.foreachRDD(lambda rdd: find_trending_topics(rdd))

In [68]:
ssc.start()

[('Sorry', 25), ('Israel', 20), ('Great', 16), ('UKIP', 16), ('America', 12), ('Wow', 11), ('Christmas', 9), ('Lib Dems', 9), ('THE', 9), ('Mexico', 9)]
[('Israel', 53), ('US', 46), ('Fnatic', 36), ('Obama', 31), ('Reddit', 30), ('OP', 29), ('Canada', 25), ('Great', 25), ('Sorry', 24), ('Bob', 24)]
[('Israel', 46), ('US', 40), ('Sorry', 40), ('Reddit', 29), ('American', 27), ('Canada', 24), ('ImGoingToHellForThis', 23), ('Great', 23), ('English', 21), ('Texas', 21)]
[('Israel', 43), ('American', 43), ('US', 42), ('Sorry', 29), ('Reddit', 28), ('Google', 27), ('Obama', 26), ('OP', 26), ('Great', 23), ('Good', 23)]
[('US', 49), ('Sorry', 43), ('Good', 25), ('Israel', 24), ('Reddit', 23), ('America', 22), ('Obama', 22), ('OP', 21), ('Wow', 20), ('Haha', 20)]
[('Reddit', 48), ('Israel', 38), ('US', 33), ('Sorry', 28), ('OP', 28), ('America', 27), ('God', 24), ('Good', 24), ('American', 23), ('Ah', 21)]
[('Fnatic', 40), ('US', 36), ('Sorry', 29), ('Great', 24), ('Please', 24), ('Reddit', 24

In [69]:
ssc.stop(stopSparkContext=False)

## Analysis

The named entities approach to finding trending topics is much more accurate than the frequent words approach. I streamed data from December 2012 and I see Christmas as a trending topic several times which seems accurate. The 2012 Israeli operation in the Gaza Strip was also around this time and there were heightened tensions in the conflict so it makes sense to see Palestine, Israel, and Hamas as trending topics. Obama is also trending topic and this makes sense giving that he won the 2012 Presidential Election a month prior. America and USA are also common trending topics which makes sense because Reddit is an American website.